In [1]:
import json
import folium
import folium.plugins
import random
import sys
from pyproj import Geod

COLORS = [
    'red', 'blue', 'green', 'purple', 'orange', 'darkred',
    'darkblue', 'darkgreen', 'cadetblue',
    'darkpurple', 'white', 'pink',
    'gray', 'black'
    ]

def delta(val=0.001):
    return random.random() * val - val/2

def get_trucks_group(json_data, marker_func):
    group = folium.FeatureGroup(name="Trucks")
    trucks = json_data["Trucks"]
    colors = {}
    for truck in trucks:
        lat = float(truck["Latitude"])
        lon = float(truck["Longitude"])
        type_name = truck["Type"]
        colors[type_name] = colors.get(type_name, COLORS[len(colors)])
        group.add_child(marker_func(lat, lon, type_name, colors[type_name]))
    return group

def get_containers_group(json_data, marker_func):
    group = folium.FeatureGroup(name="Containers")
    containers = json_data["Containers"]
    colors = {}
    for container in containers:
        lat = float(container["Latitude"])
        lon = float(container["Longitude"])
        type_name = container["Type"]
        colors[type_name] = colors.get(type_name, COLORS[len(colors)])
        group.add_child(marker_func(lat, lon, type_name, colors[type_name]))
    return group

def get_landfills_group(json_data, marker_func):
    group = folium.FeatureGroup(name="Landfills")
    lanfills = json_data["Landfills"]
    colors = {}
    for landfill in lanfills:
        lat = float(landfill["Latitude"])
        lon = float(landfill["Longitude"])
        type_name = "Lanfill"
        colors[type_name] = colors.get(type_name, COLORS[len(colors)])
        group.add_child(marker_func(lat, lon, type_name, colors[type_name]))
    return group


def truck_marker(lat, long, type_name, color):
    return folium.Marker(
        location=(lat + delta(), long + delta()),
        tooltip=type_name,
        icon=folium.Icon(color=color, icon='truck', prefix='fa')
        )

def container_marker(lat, long, type_name, color):
    return folium.CircleMarker(
        location=(lat, long),
        tooltip=type_name,
        radius=4,
        color=color
        )

def landfill_marker(lat, long, type_name, color):
    return folium.Marker(
        location=(lat + delta(), long + delta()),
        tooltip=type_name,
        icon=folium.Icon(color=color, icon='recycle', prefix='fa')
        )

def get_routes(json_data):
    route_groups = []
    geodesic = Geod(ellps='WGS84')
    routes = json_data["Routes"]
    for i, route in enumerate(routes):
        color = COLORS[i % len(COLORS)]
        group = folium.FeatureGroup(name=f"Route {i+1} ({color})")
        route_groups.append(group)
        locations = [(point["Latitude"], point["Longitude"]) for point in route["Points"]]
        route_points = folium.PolyLine(
                locations=locations, 
                weight=2,
                color=color
        )
        group.add_child(route_points)
        for j in range(1, len(locations)):
                if ((locations[j][1] - locations[j-1][1]) ** 2 + (locations[j][0] - locations[j-1][0]) ** 2) ** 0.5 < 0.00001:
                    continue
                rotation = geodesic.inv(locations[j][1], locations[j][0], locations[j-1][1], locations[j-1][0])[0] + 90

                loc = [
                    (locations[j][0] + locations[j-1][0]) / 2,
                    (locations[j][1] + locations[j-1][1]) / 2
                    ]

                triangle = folium.RegularPolygonMarker(
                    color=color,
                    fill=True,
                    fillcolor=color,
                    location=loc,
                    number_of_sides=3,
                    radius=5,
                    rotation=rotation
                    )
                group.add_child(triangle)
        route_groups.append(group)
    return route_groups



def create_map(json_file, output):
    with open(json_file) as f:
        data = json.load(f)
        
    lat = sum([cont["Latitude"] for cont in data["Containers"]]) / len(data["Containers"])       
    lon = sum([cont["Longitude"] for cont in data["Containers"]]) / len(data["Containers"])
    map_obj = folium.Map(location=(lat, lon), zoom_start=11)

    map_obj.add_child(get_trucks_group(data, marker_func=truck_marker))
    map_obj.add_child(get_containers_group(data, marker_func=container_marker))
    map_obj.add_child(get_landfills_group(data, marker_func=landfill_marker))
    for route_group in get_routes(data):
        map_obj.add_child(route_group)
    map_obj.add_child(folium.map.LayerControl('topleft', collapsed=False))

    loc = f'Total cost: {data["TotalDistance"]}'
    title_html = f'''
                 <h3 align="center" style="font-size:16px"><b>{loc}</b></h3>
                 '''

    map_obj.get_root().html.add_child(folium.Element(title_html))

    map_obj.save(output)


In [88]:
import os

PROGRAM_PATH = "./smit-opt"

def run(params):
    args = [f"--{name}={value}" for name, value in params.items()]
    command = f"{PROGRAM_PATH} {' '.join(args)}"
    res = os.system(command)
    if res != 0:
        print("ERROR")
        print("Command:", command)
        print("RES:", res)
        
params = {
    "input": "data/Amsterdam/input.json",
    "config": "data/Amsterdam/config.json",
    "output": "data/Amsterdam/output.json",
    "real": True,
    "map_file": "data/Amsterdam/osrm/netherlands-latest.osrm",
    "search": "TABU"
}

In [89]:
run(params)

ERROR
Command: ./smit-opt --input=data/Amsterdam/input.json --config=data/Amsterdam/config.json --output=data/Amsterdam/output.json --real=True --map_file=data/Amsterdam/osrm/netherlands-latest.osrm --search=TABU
RES: 2


In [3]:
create_map("results_lviv/output_0.json", "test.html")

In [6]:
with open("lviv_results/output_10.json") as f:
    data = json.load(f)
data

{'BaseSolution': 'CLUSTER',
 'Containers': [{'Latitude': 49.80594253540039,
   'Longitude': 23.988576889038086,
   'Type': 'RegularContainer'},
  {'Latitude': 49.8032341003418,
   'Longitude': 23.996932983398438,
   'Type': 'RegularContainer'},
  {'Latitude': 49.803958892822266,
   'Longitude': 23.996965408325195,
   'Type': 'RegularContainer'},
  {'Latitude': 49.80763626098633,
   'Longitude': 23.990509033203125,
   'Type': 'RegularContainer'},
  {'Latitude': 49.80380630493164,
   'Longitude': 23.9984073638916,
   'Type': 'RegularContainer'},
  {'Latitude': 49.799415588378906,
   'Longitude': 24.014829635620117,
   'Type': 'RegularContainer'},
  {'Latitude': 49.827354431152344,
   'Longitude': 24.00619125366211,
   'Type': 'RegularContainer'},
  {'Latitude': 49.80091857910156,
   'Longitude': 23.987136840820312,
   'Type': 'RegularContainer'},
  {'Latitude': 49.802711486816406,
   'Longitude': 23.98552703857422,
   'Type': 'RegularContainer'},
  {'Latitude': 49.77351379394531,
   'Lon